In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import dash
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
import json

In [2]:

class analise():
    def juntaDados(self,file):
        self.dados_DF=pd.read_csv(file[0],sep=";",index_col=['data'],parse_dates=['data'])
        for i in range(1,len(file)):
            dado=pd.read_csv(file[i],sep=";",index_col=['data'],parse_dates=['data'])
            self.dados_DF=pd.concat([dado,self.dados_DF])
    
        df_states=self.dados_DF[(~self.dados_DF["estado"].isna()) & (self.dados_DF["codmun"].isna())] #selecionando os dados dos estados
        df_states.to_csv("df_states.csv") # gerando um cvs com esses dados
        df_brasil=self.dados_DF[self.dados_DF["regiao"]=="Brasil"] #selecionando os dados so do brasil
        df_brasil.to_csv("df_brasil.csv")

    def casosDia(self):
        self.casosDia_df=self.dados_DF[['casosNovos']]
        self.roll=self.casosDia_df.rolling(window=15)
        self.rolling_mean=self.roll.mean()
        self.rolling_mean=self.rolling_mean.fillna(0)
        self.casosDia_df.plot(color='blue',figsize=(18,13),lw=0.1)
        plt.title("casos novos diarios")
        self.rolling_mean.plot(linestyle="-.",color='red',figsize=(18,13),lw=0.1)
        plt.title('media movel dos casos diarios')
        plt.show()

    def mortesDia(self):
        self.obitosDia_df=self.dados_DF[['obitosNovos']]
        self.roll=self.obitosDia_df.rolling(window=15)
        self.rolling_mean=self.roll.mean()
        self.rolling_mean=self.rolling_mean.fillna(0)
        self.obitosDia_df.plot(color='blue',figsize=(18,13),lw=0.1)
        plt.title("obtos novos diarios")
        self.rolling_mean.plot(linestyle="-.",color='red',figsize=(18,13),lw=0.1)
        plt.title('media movel dos obitos diarios')
        plt.show()

    def sazonalidas(self, data):
        for i in range(2020,2023,1):
            data[(data.index.year>=i)&(data.index.year<=i+1)].diff().plot()   

In [3]:
report=analise()

report.juntaDados(["HIST_PAINEL_COVIDBR_2020_Parte1_04mar2022.csv","HIST_PAINEL_COVIDBR_2020_Parte2_04mar2022.csv","HIST_PAINEL_COVIDBR_2021_Parte1_04mar2022.csv",
                    "HIST_PAINEL_COVIDBR_2021_Parte2_04mar2022.csv","HIST_PAINEL_COVIDBR_2022_Parte1_04mar2022.csv"])

In [ ]:
report.

In [7]:
#o arquivo json mostra as latitudes e longitudes das fronteiras de cada estado do brasil
brasil_states=json.load(open("geojson/brasil_geo.json","r"))
brasil_states["features"][0].keys()

df_states = pd.read_csv("df_states.csv")

df_states_ = df_states[df_states["data"] == "2020-05-13"]
select_columns = {"casosAcumulado": "Casos Acumulados", 
                "casosNovos": "Novos Casos", 
                "obitosAcumulado": "Óbitos Totais",
                "obitosNovos": "Óbitos por dia",
                "graficos":"report."}


app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SLATE])

fig = px.choropleth_mapbox(df_states_, locations="estado",
    geojson=brasil_states, center={"lat": -16.95, "lon": -47.78},
    zoom=4, color="casosNovos", color_continuous_scale="Redor", opacity=0.4,
    hover_data={"casosAcumulado": True, "casosNovos": True, "obitosNovos": True, "estado": True}
    )
fig.update_layout(
                paper_bgcolor="#242424",
                mapbox_style="carto-darkmatter",
                autosize=True,
                margin=go.layout.Margin(l=0, r=0, t=0, b=0),
                showlegend=False,)
df_data = df_states[df_states["estado"] == "RO"]

fig2 = go.Figure(layout={"template":"plotly_dark"})
fig2.add_trace(go.Scatter(x=df_data["data"], y=df_data["casosAcumulado"]))
fig2.update_layout(
    paper_bgcolor="#242424",
    plot_bgcolor="#242424",
    autosize=True,
    margin=dict(l=10, r=10, b=10, t=10),
    )

app.layout=dbc.Container(
        dbc.Row([
            dbc.Col([
                dcc.Graph(id="croropleth-map",figure=fig)
            ])
        ])
)

if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Mar/2022 01:44:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Mar/2022 01:44:08] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Mar/2022 01:44:08] "GET /_favicon.ico?v=1.19.0 HTTP/1.1" 200 -
127.0.0.1 - - [06/Mar/2022 01:44:10] "GET /_dash-layout HTTP/1.1" 200 -
